In [19]:
import heapq

import sys
sys.path.append("../")
from tree_search import Node
import csv
import numpy as np
import pandas as pd

In [45]:
def read_node_fron_file(file_name):
    graph = {}

    with open(file_name, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)

        # Skip the header row
        next(csv_reader)

        for row in csv_reader:
            #print(row[0])
            graph = add_to_graph(row, graph)

    return graph

def add_to_graph(row, graph):
    parent_name = row[0]
    parent_g_n = float(row[1])
    parent_coords = float(row[2]),float(row[3])

    child_name = row[4]
    child_g_n = float(row[5])
    child_coords = float(row[6]), float(row[7])

    #check if the parent is alreay exist
    if (parent_name in graph):
        parent_node = graph[parent_name]
    else:
        parent_node = Node(parent_name, parent_coords, None, parent_g_n, [])
        graph[parent_node.name] = parent_node
        
    child_node = Node(child_name, child_coords, parent_node, child_g_n,[])
    parent_node.children.append(child_node)

    graph[child_node.name] = child_node

    return graph

In [46]:
def heuristic(node, goal_state):
    
    # distance to energy

    distance = np.linalg.norm(np.array(node.coords) - np.array(goal_state.coords))
    energy = distance * 1000

    return energy


In [99]:
def astar(start_state, goal_state, search_tree):
    # Initialize the open and closed sets
    open_set = [(0, start_state)]
    closed_set = set()
    
    # Initialize the dictionary to keep track of the path and cost
    path = {}
    cost = {}
    path[start_state] = None
    cost[start_state] = 0
    
    # Loop until the open set is empty
    while len(open_set) > 0:
        # Pop the node with the smallest f value from the open set
        current_node = heapq.heappop(open_set)[1]
        print(current_node)
        current_node = search_tree[current_node].name
        
        # Check if the current node is the goal state
        if current_node == goal_state:
            # Construct the path and return it along with the total cost
            total_cost = cost[current_node]
            current_path = []
            while current_node is not None:
                current_path.append(current_node)
                current_node = path[current_node]
            current_path.reverse()
            return current_path, total_cost
        
        # Add the current node to the closed set
        closed_set.add(current_node)
        
        # Loop through the neighbors of the current node
        # print(type(current_node))
        # print(search_tree[current_node].children)
        for neighbor in search_tree[current_node].children:
            edge_cost = neighbor.g_n
            neighbor = neighbor.name
            print(neighbor)
            # Skip neighbors that have already been visited
            if neighbor in closed_set:
                continue
            
            # Calculate the tentative g and f values for the neighbor
            tentative_g = cost[current_node] + edge_cost
            tentative_f = tentative_g + heuristic(search_tree[neighbor], search_tree[goal_state])
            
            # Add the neighbor to the open set if it's not already there
            if (tentative_f, neighbor) not in open_set:
                heapq.heappush(open_set, (tentative_f, neighbor))
                
            # Update the path and cost if the tentative g value is better than the current one
            if tentative_g < cost.get(neighbor, float('inf')):
                path[neighbor] = current_node
                cost[neighbor] = tentative_g
                
    # If we reach this point, there is no path from the start to the goal state
    return None, None


In [82]:
for neighbor in campus_map["SU"].children:
    print(neighbor)

In [100]:
for bot in botAssignments:
    overall_path = []
    total_cost = 0

    assignments = botAssignments[bot]

    if ( (len(assignments) == 2) & (assignments[0]["Location"] == assignments[1]["Location"]) ) | (len(assignments) == 1):
        path, cost = astar(start_state, assignments[0]["Location"], campus_map)
        overall_path.append(path)
        total_cost += cost

        path, cost = astar(assignments[0]["Location"], start_state, campus_map)
        overall_path.append(path)
        total_cost += cost

    else:
        path, cost = astar(start_state, assignments[0]["Location"], campus_map)
        overall_path.append(path)
        total_cost += cost

        path, cost = astar(assignments[0]["Location"], assignments[1]["Location"], campus_map)
        overall_path.append(path)
        total_cost += cost

        path, cost = astar(assignments[1]["Location"], start_state, campus_map)
        overall_path.append(path)
        total_cost += cost
    
    print("Bot ", bot, " Used ", total_cost, " Energy Units")
    print("Bot ", bot, " Traveled the Following Path:")
    print(overall_path)

SU
SU
SIE
MME
2ndStG
2ndStG
SIE
ChBld
MME
YH
SofI
S&Hsci
ChBld
YH
YH


TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [47]:
start_state = "SU"

campus_map = read_node_fron_file("../graph_data")
campus_map.keys()

dict_keys(['SU', 'SIE', 'MME', '2ndStG', 'ChBld', 'YH', 'SofI', 'S&Hsci'])

In [40]:
order1 = {"Location": "SU",
         "State": campus_map["SU"],
         "Items": ["Chickfila"]}
order2 = {"Location": "SIE",
         "State": campus_map["SIE"],
         "Items": ["Burger"]}
order3 = {"Location": "YH",
         "State": campus_map["YH"],
         "Items": ["Canes"]}
order4 = {"Location": "SofI",
         "State": campus_map["SofI"],
         "Items": ["Burrito"]}
order5 = {"Location": "SIE",
         "State": campus_map["SIE"],
         "Items": ["Panera"]}

orders = [order1, order2, order3, order4, order5]
orders

[{'Location': 'SU',
  'State': <tree_search.Node at 0x1ce59bf1db0>,
  'Items': ['Chickfila']},
 {'Location': 'SIE',
  'State': <tree_search.Node at 0x1ce59bf2f80>,
  'Items': ['Burger']},
 {'Location': 'YH',
  'State': <tree_search.Node at 0x1ce59bf35e0>,
  'Items': ['Canes']},
 {'Location': 'SofI',
  'State': <tree_search.Node at 0x1ce59bf2e90>,
  'Items': ['Burrito']},
 {'Location': 'SIE',
  'State': <tree_search.Node at 0x1ce59bf2f80>,
  'Items': ['Panera']}]

Assignments

In [41]:
botAssignments = {}

for bot in ["Bot1", "Bot2", "Bot3"]:
    botAssignments[bot] = []

    for order in orders:
        locations = [order["Location"] for order in orders]

        if locations.count(order["Location"]) >= 2:

            same_location_orders = [o for o in orders if order["Location"] == o["Location"]]

            botAssignments[bot] = same_location_orders
            orders.remove(same_location_orders[0])
            orders.remove(same_location_orders[1])

        else:
            botAssignments[bot].append(order)
            orders.remove(order)

        if len(botAssignments[bot]) == 2:
            break

print(botAssignments)

{'Bot1': [{'Location': 'SU', 'State': <tree_search.Node object at 0x000001CE59BF1DB0>, 'Items': ['Chickfila']}, {'Location': 'YH', 'State': <tree_search.Node object at 0x000001CE59BF35E0>, 'Items': ['Canes']}], 'Bot2': [{'Location': 'SIE', 'State': <tree_search.Node object at 0x000001CE59BF2F80>, 'Items': ['Burger']}, {'Location': 'SIE', 'State': <tree_search.Node object at 0x000001CE59BF2F80>, 'Items': ['Panera']}], 'Bot3': [{'Location': 'SofI', 'State': <tree_search.Node object at 0x000001CE59BF2E90>, 'Items': ['Burrito']}]}


Delivery